In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import numpy as np
import tensorflow as tf
from glob import glob

import models
from matplotlib import pyplot as plt

In [4]:
folders = glob('./dataset/summer2winter_yosemite/*')

In [5]:
data_A_paths = glob('./dataset/summer2winter_yosemite/trainA/*')
data_B_paths = glob('./dataset/summer2winter_yosemite/trainB/*')

In [6]:
model = models.CycleGan()

In [7]:
imgA = plt.imread(data_A_paths[12]).reshape(1, 256, 256, 3)/127.5-1
imgB = plt.imread(data_B_paths[32]).reshape(1, 256, 256, 3)/127.5-1

In [ ]:
%%time
model.train(data_A_paths[0:1], data_B_paths[0:1], batch_size=1, epoch=100)

In [10]:
inA = tf.placeholder(tf.float32, [None, 256, 256, 3], name='inA')
inB = tf.placeholder(tf.float32, [None, 256, 256, 3], name='inB')
genB = models.generator('genA2B', inA, False)
genA = models.generator('genB2A', inB, False)

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    genedB = sess.run(genB, feed_dict={inA: imgA})
    genedA = sess.run(genA, feed_dict={inB: genedB})

In [19]:
abs(genedA - imgA).mean()

0.5811324417866809

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(t, feed_dict={model.input_real_A: imgA, model.input_real_B: imgB})
    print(x.shape)

(3, 3, 32, 256)


In [10]:
l7 = None
for t in tf.global_variables():
    if ('discB/l7/kernel:0' in t.name):
        print(t)
        l7 = t


<tf.Variable 'discB/l7/kernel:0' shape=(3, 3, 256, 256) dtype=float32_ref>


In [2]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./logs')
    writer.add_graph(sess.graph)

In [8]:
for t in tf.trainable_variables():
    if t.name == 'discB/l7/kernel:0':
        break;